In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmcrameri import cm

import os
from pathlib import Path
import shutil
import gzip
import re

import timeit

In [2]:
# data_path = 'data/simulation_data'
# output_file = 'processed_data/WDBH_data.csv'
# header = "#N,#rv,#rg,#z,#t_snapshot[myr],#M1[MSUN],#M2[MSUN],#k1,#k2,#id1,#id2,#sma[AU],#ecc,#bin_star_radius0[RSUN],#bin_star_radius1[RSUN],#snapshot, #roche_lobe1_calc[RSUN], #roche_lobe2_calc[RSUN],#radrol0,#radrol1"


In [3]:
data_path = 'data/updated_simulation_data'
output_file = 'processed_data/updated_WDBH_data.csv'
header = "#N,#rv,#rg,#z,#t_snapshot[myr],#M1[MSUN],#M2[MSUN],#k1,#k2,#id1,#id2,#sma[AU],#ecc,#bin_star_radius0[RSUN],#bin_star_radius1[RSUN],#snapshot, #roche_lobe1_calc[RSUN], #roche_lobe2_calc[RSUN],#radrol0,#radrol1"


In [4]:
### move the misc files to a new folder
# Loop over all model folders
for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    if os.path.isdir(folder_path):
        # Loops through all the snapshots
        for subfolder in os.listdir(folder_path):
            subfolder_path = os.path.join(folder_path, subfolder)
            
            infosubfolder_path = os.path.join(folder_path, 'info')
            if not os.path.exists(infosubfolder_path):
                os.makedirs(infosubfolder_path)

            if Path(subfolder_path).is_file() and Path(subfolder_path).suffix == '.log':
                # move the log file to the info folder
                shutil.move(subfolder_path, infosubfolder_path)
            if Path(subfolder_path).is_file() and Path(subfolder_path).suffix == '.sh':
                # move the sh file to the info folder
                shutil.move(subfolder_path, infosubfolder_path)
            if Path(subfolder_path).is_file() and Path(subfolder_path).suffix == '.dat' and not Path(subfolder).stem.startswith("initial.bhinfo"):
                # move the dat file to the info folder
                shutil.move(subfolder_path, infosubfolder_path)
            if Path(subfolder_path).is_file() and Path(subfolder_path).suffix == '.gz':
                print(subfolder_path)

In [5]:
### unzip everything
# Loop over all model folders
for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    if os.path.isdir(folder_path):
        # Loops through all the snapshots
        for subfolder in os.listdir(folder_path):
            subfolder_path = os.path.join(folder_path, subfolder)
            if Path(subfolder_path).is_file() and Path(subfolder_path).suffix == '.gz':
                # Unzip the .gz archive
                with gzip.open(subfolder_path, 'rb') as f_in:
                    dat_file_path = os.path.join(folder_path, Path(subfolder).stem)
                    with open(dat_file_path, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                # Delete the .gz archive after extraction
                os.remove(subfolder_path)
                print(f"Extracted and copied {dat_file_path} and deleted {subfolder_path}")

In [ ]:
### move the misc files to a new folder
# Loop over all model folders
for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    if os.path.isdir(folder_path):
        # Loops through all the snapshots
        for subfolder in os.listdir(folder_path):
            subfolder_path = os.path.join(folder_path, subfolder)
            
            infosubfolder_path = os.path.join(folder_path, 'info')
            if not os.path.exists(infosubfolder_path):
                os.makedirs(infosubfolder_path)

            if Path(subfolder_path).is_file() and Path(subfolder_path).suffix == '.log':
                # move the log file to the info folder
                shutil.move(subfolder_path, infosubfolder_path)
            if Path(subfolder_path).is_file() and Path(subfolder_path).suffix == '.sh':
                # move the sh file to the info folder
                shutil.move(subfolder_path, infosubfolder_path)
            if Path(subfolder_path).is_file() and Path(subfolder_path).suffix == '.dat' and not Path(subfolder).stem.startswith("initial.bhinfo"):
                # move the dat file to the info folder
                shutil.move(subfolder_path, infosubfolder_path)
            if Path(subfolder_path).is_file() and Path(subfolder_path).suffix == '.gz':
                print(subfolder_path)

In [6]:
def rl_rad(q, a):
    return 215.032*a*0.49*q**(2/3)/(0.6*q**(2/3)+np.log(1.0+q**(1/3)))

In [7]:

id_WD = [10, 11, 12]
id_BH = [14]

all_WDBH = []


### Grab all values
# Loop over all model folders
for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder);
    # Checks is a folder (to prevent funny things from happening)
    if os.path.isdir(folder_path):
        # Loops through all the snapshots
        for subfolder in os.listdir(folder_path):
            subfolder_path = os.path.join(folder_path, subfolder);
            
            ### Find the timeunitsmyr conversion value
            if os.path.isdir(subfolder_path):
                for infosubfolder in os.listdir(subfolder_path):
                    if infosubfolder.endswith('.sh'):
                        infosubfolder_path = os.path.join(subfolder_path, infosubfolder);
                        # Read the file
                        with open(infosubfolder_path, 'r') as file:
                            lines = file.readlines();

                        # Extract the value for timeunitsmyr
                        for line in lines:
                            if line.startswith('timeunitsmyr'):
                                _, value = line.split('=');
                                timeunitsmyr = float(value.strip());
                                print(timeunitsmyr);
                                break
            
            ### Obtain desired data from info files
            #if Path(subfolder_path).is_file() and Path(subfolder_path).suffix == '.dat':
            if subfolder.endswith('.dat'):
                # Check if file is bhinfo
                if Path(subfolder).stem.startswith("initial.bhinfo"):
                    ### Grab the values of the parameters from the file path
                    # Define the regular expression pattern
                    pattern = r"N(?P<N>[\d.]+)_rv(?P<rv>[\d.]+)_rg(?P<rg>[\d.]+)_z(?P<z>[\d.]+)";
                    # Search for the pattern in the file path
                    match = re.search(pattern, subfolder_path);
                    # If a match is found, extract the values and convert them to floats
                    if match:
                        mod_params = [float(match.group('N')), float(match.group('rv')), float(match.group('rg')), float(match.group('z'))];
                    else:
                        print("Pattern not found in the file path.");

                    # Grab just the final file name from subfolder_path
                    file_name = os.path.splitext(os.path.basename(subfolder_path))[0];

                    ### Read data from files
                    # Read the first line of the file as a string
                    with open(subfolder_path, 'r') as file:
                        first_line = file.readline().strip();
                        # Extract the float value after 't=' in the first_line string
                        match = re.search(r"t=([\d.]+)", first_line);
                        if match:
                            t_value = timeunitsmyr * float(match.group(1));
                        else:
                            t_value = 'nan';
                        # Append t_value to the mod_params list
                        mod_params.append(t_value);

                    # Load the data from a file
                    data_array = np.genfromtxt(subfolder_path, comments="#");

                    # Skip over the files that are empty
                    if list(data_array.shape)[-1] == 62:
                        # Check if the list is a list of lists. If not, make it one. 
                        if all(isinstance(sublist, list) for sublist in data_array.tolist()):
                            data_list = data_array.tolist();
                        else:
                            data_list = [data_array.tolist()];
                        
                        # Grab the binflags
                        binflags = np.array(data_list)[:, 7].tolist();

                        # Check binflags
                        check_bin = np.array([flag == 1 for flag in binflags]);
                        events_bin = np.array(data_list)[check_bin];
                        print(events_bin[:, 17])
                        # Check WDBH
                        check_WDBH = ((np.isin(events_bin[:, 17], id_BH) & np.isin(events_bin[:, 18], id_WD)) | (np.isin(events_bin[:, 18], id_BH) & np.isin(events_bin[:, 17], id_WD)));
                        events_WDBH = events_bin[check_WDBH];

                        # Calculate Roche lobe radius
                        q1 = events_WDBH[::, 8].astype(float) / events_WDBH[::, 9].astype(float);
                        rl_rad1 = rl_rad(q1, events_WDBH[::, 12].astype(float));

                        q2 = 1 / q1
                        rl_rad2 = rl_rad(q2, events_WDBH[::, 12].astype(float));
                        
                        # Grab the data we want
                        events_WDBH_data = np.array([events_WDBH[::, 8].astype(float), events_WDBH[::, 9].astype(float), events_WDBH[::, 17].astype(float), events_WDBH[::, 18].astype(float), events_WDBH[::, 10].astype(float), events_WDBH[::, 11].astype(float), events_WDBH[::, 12].astype(float), events_WDBH[::, 13].astype(float), events_WDBH[::, 21].astype(float), events_WDBH[::, 22].astype(float), np.full(len(events_WDBH), file_name), rl_rad1, rl_rad2, events_WDBH[::, 43].astype(float), events_WDBH[::, 44].astype(float)]).T;

                        # Append the model parameters to the start of each sublist
                        data_list = [mod_params + sublist if type(sublist) is list else mod_params + [sublist] for sublist in events_WDBH_data.tolist()];
                        
                        # Append the data to the list of all data
                        if len(data_list) > 0:
                            all_WDBH += data_list;

#print(all_WDBH)

884.472
[ 0. 14.  0. 14.]
[ 0. 14. 14. 14.  0.  0. 14.  0. 14.  0. 14. 14. 14. 14.  1. 14.  0. 14.
 14.]
[14. 14. 14.  0. 14.  0. 14.  1. 14. 14. 14. 14.  0.  0. 14.  0.  0. 14.
 14.  0. 14. 14. 14.  0.  0. 14.  0. 14. 14. 14. 14. 14. 14. 14. 14.  0.
 14. 14.  1. 14.]
[ 0. 14. 14. 14.  0.  0. 14. 14.  0. 14.  0.  0. 14. 14. 14. 14. 14. 14.
 14.  0.  0. 14.  0.  1.  0. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14.
  0. 14. 14.  0.  0. 14. 14. 14.  0. 14. 14.  1. 14.]
[14. 14.  0. 14. 14. 14.  0.  0.  0. 14.  0.  0. 14. 14. 14. 14. 14.  0.
 14.  0. 14. 14. 14. 14. 14. 14. 14.  0.  1. 14.  0. 14. 14.  0. 14. 14.
 14.  0.  0. 14. 14. 14.  0.  0.  0. 14. 14. 14.  1.  0. 14.]
[14. 14. 14.  0. 14.  0.  0.  0.  0.  0. 14.  0.  0.  0. 14. 14.  0. 14.
  0. 14. 14.  0. 14.  0. 14. 14. 14. 14. 14.  1. 14. 14.  1. 14. 14.  1.
  0.  0. 14. 14. 14. 14. 14. 14. 14. 14.  0. 14. 14. 14. 14.  0. 14.]
[14. 14. 14.  0. 14. 14.  0.  0.  1. 14.  0. 14.  1. 14. 14.  0. 14. 14.
  0.  0. 14. 14. 14.  1. 14. 14. 1

C:\Users\willi\AppData\Local\Temp\ipykernel_57560\497776682.py:67: UserWarning: genfromtxt: Empty input file: "data/updated_simulation_data\N16_rv0.5_rg2.0_z0.002\initial.bhinfo0000.dat"
  data_array = np.genfromtxt(subfolder_path, comments="#");
C:\Users\willi\AppData\Local\Temp\ipykernel_57560\497776682.py:67: UserWarning: genfromtxt: Empty input file: "data/updated_simulation_data\N16_rv0.5_rg2.0_z0.002\initial.bhinfo0001.dat"
  data_array = np.genfromtxt(subfolder_path, comments="#");
C:\Users\willi\AppData\Local\Temp\ipykernel_57560\497776682.py:67: UserWarning: genfromtxt: Empty input file: "data/updated_simulation_data\N16_rv0.5_rg2.0_z0.002\initial.bhinfo0002.dat"
  data_array = np.genfromtxt(subfolder_path, comments="#");
C:\Users\willi\AppData\Local\Temp\ipykernel_57560\497776682.py:67: UserWarning: genfromtxt: Empty input file: "data/updated_simulation_data\N16_rv0.5_rg2.0_z0.002\initial.bhinfo0003.dat"
  data_array = np.genfromtxt(subfolder_path, comments="#");


[14.  0.  0.  0.  0.  0. 14.  0.  0. 14. 14.  0.  0.  0. 14.  1.  0.  1.
 14. 14. 14. 14.  0.  0. 14.  1. 14. 14.  1. 14. 14. 14. 14. 14.  0.  0.
  0.  0. 14. 14.  0. 14. 14.  0. 14.  0.  0.  0. 14. 14. 14. 14.  0. 14.
 14. 14. 14. 14. 14.  0. 14.  0. 14. 14. 14. 14.]
[14. 14.  0.  0. 14. 14.  0.  0. 14.  1. 14. 14.  0. 14. 14.  1. 14. 14.
  0.  0. 14. 14. 14. 14.  1. 14. 14.  0.  0.  0.  0.  0. 14.  0.  0. 14.
 14. 14.  0. 14. 14. 14.  0.  0. 14.  1. 14. 14. 14. 14. 14. 14.  0.  0.
  0. 14.  0. 14.  0.  0. 14.  0.  0. 14. 14.  0. 14. 14. 14. 14. 14.  0.
  0. 14.  1.  0. 14. 14.  1. 14.  0. 14.  1. 14.  0. 14.  0.]
[ 0. 14.  1.  0.  0.  0.  0. 14.  0.  0. 14. 14. 14.  0.  1.  1.  1. 14.
  0. 14.  0. 14.  1.  0. 14.  0.  0. 14. 14.  0.  0.  0.  0.  0.  1.  0.
 14. 14. 14. 14.  0. 14. 14. 14. 14. 14. 14.  0.  0. 14. 14. 14.  0.  1.
 14. 14. 14. 14. 14.  0.  1. 14.  0. 14.  0.  0. 14.  1.  0.  0.  0. 14.
 14. 14.  0. 14.  0. 14. 14. 14. 14. 14.  0.  0. 14.  0.  0. 14. 14. 14.
  0. 14. 14.

C:\Users\willi\AppData\Local\Temp\ipykernel_57560\497776682.py:67: UserWarning: genfromtxt: Empty input file: "data/updated_simulation_data\N16_rv0.5_rg2.0_z0.02\initial.bhinfo0000.dat"
  data_array = np.genfromtxt(subfolder_path, comments="#");
C:\Users\willi\AppData\Local\Temp\ipykernel_57560\497776682.py:67: UserWarning: genfromtxt: Empty input file: "data/updated_simulation_data\N16_rv0.5_rg2.0_z0.02\initial.bhinfo0001.dat"
  data_array = np.genfromtxt(subfolder_path, comments="#");
C:\Users\willi\AppData\Local\Temp\ipykernel_57560\497776682.py:67: UserWarning: genfromtxt: Empty input file: "data/updated_simulation_data\N16_rv0.5_rg2.0_z0.02\initial.bhinfo0002.dat"
  data_array = np.genfromtxt(subfolder_path, comments="#");


[ 0.  0.  0.  0.  0.  0.  0. 14. 14. 14.  0. 14.  0.  0. 14.]
[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0. 14.  0. 14.  0.  0.  0. 14.  0.
  0. 14.]
[ 0. 14.  1.  1.  0.  0.  0.  0.  0.  0.  0. 14.  0. 14.  3.  0.  0.  0.
 14.  0.  0. 14.  0.  0. 14. 14.]
[14.  0.  0.  1.  0.  1.  1.  1.  0. 14.  0.  0.  1.  0.  0.  1.  0.  0.
 14. 14. 14. 14.  0. 14. 14.  0.  1. 14.  0.  0.  0. 14.]
[ 0. 14.  0. 14. 14.  0.  0.  1.  1.  0.  1.  0.  0.  0. 14.  1.  1.  1.
  0.  0.  0. 14.  1.  0.  1.  0. 14.]
[14. 14.  0. 14.  4.  0.  4. 14.  1.  0. 14.  0.  0.  1. 14.  0.  0.  0.
  0.  0.  0.  0. 14.]
[ 0. 14. 14.  1.  0.  0. 14.  0. 14.  1.  0. 14. 14.  0. 14. 14.  4.  4.
  0. 14. 14.  0.  0.]
[ 1.  0. 14. 14. 14.  0.  0.  0. 14.  1.  0.  1.  0. 14.  0.  0.  0. 14.
 14.  0. 14.]
[14. 14.  0.  0.  1. 14.  0.  0.  0. 14. 14.  0.  0. 14. 14.  0. 14.  0.
  1.]
[14. 14.  0.  1.  0.  0.  0.  0. 14.  0. 14.  0. 14. 14. 14.  0.  0.  0.
 14. 14.]
[14.  0.  0. 14. 14.  0.  1.  0.  0. 14. 14. 14.  0. 14.  0.  0. 14

...

[14.  0.  0. 14.  0.  0. 14. 14. 14.  0.  0. 14. 14. 14. 14. 14.  0.  0.
 14. 14.  0.  0.  0.  0.  0.  0. 14.  0. 14. 14.  0.  0.  0. 14.  1.  0.
  0.  0. 14.  0.  0. 14.  0. 14.  0.  0.  0. 14. 14.  1.  0.  1. 14.  0.
  1.  0. 14. 14. 14.  0.  0.  0. 14. 14.  0. 14. 14. 14. 14. 14.  0. 14.
  1. 14. 14. 14. 14. 14.]
[14.  0. 14.  0.  0. 14.  0.  0.  0. 14. 14.  0. 14.  0.  0.  0.  0.  0.
 14.  1. 14. 14. 14. 14.  0.  0.  0.  0. 14.  0.  0. 14.  0.  1.  0. 14.
  0. 14. 14.  0. 14. 14. 14. 14.  0. 14. 14.  0.  0. 14.  0. 14.  0. 14.
  0.  1. 14. 14.  0. 14. 14. 14. 14.  0.  1. 14. 14.  0.  0.  0. 14. 14.
  0. 14.  0.  1.  0. 14. 14.]
[14.  0.  0.  0. 14. 14. 14. 14.  0. 14.  0. 14. 14.  0. 14.  0. 14.  0.
  0. 14.  0.  0.  0.  0. 14.  0.  0.  0.  0.  0. 14.  0.  0. 14.  0. 14.
  1. 14.  0.  0.  0.  0. 14.  0.  0. 14. 14. 14.  0.  1.  1.  0.  0. 14.
  0. 14. 14. 14. 14.  0. 14.  0.  0.  0.  0. 14.  1.  0. 14.  0.  0. 14.
  0.  0. 14.  1.  0. 14. 14. 14.  0. 14.  0.  1. 14. 14. 14.  0. 14.

C:\Users\willi\AppData\Local\Temp\ipykernel_57560\497776682.py:67: UserWarning: genfromtxt: Empty input file: "data/updated_simulation_data\N8.0_rv0.5_rg8.0_z0.002\initial.bhinfo0164.dat"
  data_array = np.genfromtxt(subfolder_path, comments="#");
C:\Users\willi\AppData\Local\Temp\ipykernel_57560\497776682.py:67: UserWarning: genfromtxt: Empty input file: "data/updated_simulation_data\N8.0_rv0.5_rg8.0_z0.002\initial.bhinfo0165.dat"
  data_array = np.genfromtxt(subfolder_path, comments="#");
C:\Users\willi\AppData\Local\Temp\ipykernel_57560\497776682.py:67: UserWarning: genfromtxt: Empty input file: "data/updated_simulation_data\N8.0_rv0.5_rg8.0_z0.002\initial.bhinfo0166.dat"
  data_array = np.genfromtxt(subfolder_path, comments="#");
C:\Users\willi\AppData\Local\Temp\ipykernel_57560\497776682.py:67: UserWarning: genfromtxt: Empty input file: "data/updated_simulation_data\N8.0_rv0.5_rg8.0_z0.002\initial.bhinfo0167.dat"
  data_array = np.genfromtxt(subfolder_path, comments="#");
C:\Users

[14. 14. 14.  0.  0. 14. 14. 14.  0.  1.  0.  1.  0.  0. 14.  0.  0. 14.
  1.  0.  0. 14. 14. 14.]
[14. 14. 14. 14. 14.  0.  0.  0. 14. 14.  0.  1.  0.  0. 14.  1. 14.  0.
 14.  1. 14.  1. 14. 14.]
[14.  0.  0. 14.  0. 14.  0.  0.  1.  1. 14. 14. 14. 14. 14. 14.  1. 14.
  0.  1. 14.]
[14. 14. 14.  1.  0.  0. 14. 14. 14.  0. 14.  1.  0. 14. 14. 14.]
[ 0. 14.  1. 14.  1.  1. 14.  0.  0. 14.  0. 14. 14. 14.]
[ 0.  0. 14. 14.  0. 14.  1. 14. 14. 14.  1.  0. 14. 14.]
[14.  1.  0. 14. 14. 14.  0. 14.  1. 14.  0. 14. 14.  0. 14.]
[14. 14. 14. 14. 14.  0. 14.  0. 14.  1.  0. 14. 14. 14.]
[14. 14. 14. 14. 14.  1.  0.  0.  1. 14.  0. 14. 14.]
[14. 14.  1.  0. 14. 14. 14. 14.  0.  1. 14.  1.  0.  1. 14. 14.]
[ 1. 14.  1.  1. 14.  1.  1. 14.  0. 14.  0.  0.  1. 14. 14.]
[14.  0. 14.  1.  1.  1. 14.  1.  0.  0. 14. 14.]
[14. 14.  0.  0.  0.  1.  1.  1. 14. 14.]
[14. 14. 14. 14. 14.  1.  1.  0.  0.  0. 14. 14.]
[14. 14.  1.  0.  1.  0. 14. 14.  0.  1. 14. 14.]
[ 0. 14. 14. 14. 14.  1. 14.  0. 14.  0

C:\Users\willi\AppData\Local\Temp\ipykernel_57560\497776682.py:67: UserWarning: genfromtxt: Empty input file: "data/updated_simulation_data\N8.0_rv1.0_rg2.0_z0.02\initial.bhinfo0097.dat"
  data_array = np.genfromtxt(subfolder_path, comments="#");


[14. 14. 14. 14. 14. 14. 14.  0.  1.  0. 14.  0.  0. 14.  0. 14. 14. 14.
 14. 14.  0. 14. 14.  0. 14.  0.  1. 14.  0. 14.  0.  0.  0. 14.  0. 14.
 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14.  0. 14. 14.  0.  0.  0.  0.
 14. 14.  0.  0. 14. 14.  0.  0. 14. 14.  0.  0.  1.  0.  0. 14. 14.  0.
 14. 14. 14. 14. 14. 14. 14.]
[ 0. 14. 14.  0. 14. 14.  0. 14. 14.  0.  1. 14.  1.  1.  0.  0. 14.  0.
  0.  0. 14. 14. 14. 14. 14. 14.  0. 14.  0. 14.  1.  0.  1.  0. 14.  0.
  0.  0. 14. 14.  1.  0. 14. 14. 14. 14.  0.  0.  0. 14. 14. 14. 14. 14.]
[14. 14.  0.  0.  1. 14. 14. 14. 14.  1. 14. 14.  0. 14. 14.  0. 14.  1.
  0.  1.  0.  0. 14. 14. 14.  0.  0.  0.  1.  0.  0.  0. 14.  0.  0.  0.
  0.  0. 14. 14.  0. 14. 14. 14.]
[14. 14.  0.  1.  1. 14.  0.  0.  0.  0.  1.  0. 14. 14.  0.  0. 14. 14.
 14. 14.  0. 14.  1.  1.  0.  0.  0. 14.  0.  0. 14. 14. 14. 14.]
[ 0. 14.  0.  0.  0.  0.  1. 14.  1. 14. 14.  0.  0.  0.  0. 14. 14.  0.
  1.  1.  1. 14. 14.  1. 14.  0. 14.  0.  0. 14. 14. 14. 14.]
[ 1.

In [8]:
### Write the data to the output file
with open(output_file, 'w') as file:
    # Write the header
    file.write(header + '\n')

    # Write the data
    for row in all_WDBH:
        file.write(','.join(map(str, row)) + '\n')

In [9]:
# conv_path = 'data/simulation_data/N2.0_rv0.5_rg2.0_z0.002/info/initial.conv.sh'

# convs = {}

# # Match variables
# pattern = re.compile(r'^\s*(\w+)\s*=\s*([\w.+-eE]+)\s*$', re.MULTILINE)

# # Read the file
# with open(conv_path, 'r') as file:
#     content = file.read()

# # Find all matches
# matches = pattern.findall(content)
# matches.pop(0)

# # Store matches in the dictionary
# for match in matches:
#     variable, value = match
#     convs[variable] = float(value)

# # Takes keys from dictionary and assigns them as variables with the value from the dictionary
# locals().update(convs)